# Problema Abordado

# Implementação

## Importações

In [2]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

##Funções Auxiliares

In [ ]:
def train_model(
    model,          # Seu modelo
    train_loader,   # DataLoader de treino
    criterion,      # Função de perda
    optimizer,      # Otimizador
    device,         # "cuda" ou "cpu"
    epochs=10       # Número de épocas
):
    model.to(device)
    model.train()  # Modo treino

    for epoch in range(epochs):
        epoch_loss = 0.0

        for images, captions in train_loader:

            images = images.to(device)
            captions = captions.to(device)


            optimizer.zero_grad()


            outputs = model(images, captions[:, :-1])
            loss = criterion(
                outputs.view(-1, outputs.size(-1)),
                captions[:, 1:].reshape(-1)
            )

            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        # Loss médio da época
        avg_loss = epoch_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")

## Dados

###Carregando o dataset


In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),

])

100%|██████████| 26.4M/26.4M [00:01<00:00, 14.7MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 234kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 4.32MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 19.1MB/s]


### Data Loader

In [ ]:
batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

## Rede Implementada

In [ ]:


class ImageToText(nn.Module):
    def __init__(self):
        super().__init__()

        # Encoder (CNN - Custom)


        # Decoder (GPT2)


    def forward(self,):


# Treinamento da Rede

# Qualidade dos Resultados

# Discussão Geral